# ASTR 400B Research Assignment 3:
This code is a draft for what will be used for my final project.
The general topic of my research assignment is analyzing how galaxies and dark matter halos evolve together through mergers. The major question is whether the dark matter halo remnant will be prograde or retrograde, relative to the rotation of the baryon disk. This code will focus on the angular momentum calculation which will be used to analyze whether the halo is prograde or retrograde.

academic papers used for inspiration and thought:
-------------------------------------------------
Drakos+2019 (discussion of dark matter halo mergers)\
Chua+2019 (claim that with baryon matter, halos are more spherical)\
Teklu+2015 (discussion of angular momentum in disk and halo)\
Carollo+2007 (claim retrograde and prograde MW halo)\
Koppelman+2019 (claim retrograde MW halo)

Notes to self:
-----
can ignore M33 (maybe see how M33 changes the results if time allows?)\
RADIUS MUST BE SPECIFIED IN THE BELOW CODE:\
determine when disk and halo radii drop off for MW and M31 (homework 5?)\
--can use 20 kpc as a default for the disk radius\
--to determine the halo radius, find virial radius (can use 200 kpc for default)\
look at orbitCOM for using an array of times

In [ ]:
# import modules
import numpy as np
import astropy.units as u

# import plotting module
import matplotlib.pyplot as plt

# import previous code
from ReadFile import Read
from CenterofMass2 import CenterofMass
from ParticleProperties import ParticleProperties

# Angular Momentum:
To calculate the angular momentum:
\begin{equation}
    \vec{L} = \sum_{i} \vec{r_i} \times \vec{p_i} = \sum_{i} m_i (\vec{r_i} \times \vec{v_i})
\end{equation}
To answer this question, we need to analyze the direction of the angular momentum of the halo and the angular momentum of the disk. If the dark matter halo is prograde, the orbital rotation is in the same direction and if the dark matter halo is retrograde, the orbital rotation is in the opposite direction. 

In [ ]:
def AngularMomentum(galaxy, start, end, n):
    '''
        This function calculates the angular momentum of a galaxy component at a certain 
        time

        PARAMETERS:
        -----------
            galaxy: 'str'
                the name of the galaxy (MW, M31, or M33)
            start: 'int'
                the number of the first snapshot to be read in
            end: 'int'
                the number of the last snapshot to be read in
            n: 'int'
                indicates the interval over while the COM will be returned

        OUTPUT:
        -------
            AngularMomentum: np.array
                the angular momentum of the galaxy component in kg m^2/s
    '''

    # L = r x p = m * r x v
    # where the variables are coming from:
        # r: from particle properties and COM p
        # v: from particle properties and COM v
        # m: from particle properties

    # generating the snapshot id sequence
    snap_ids = np.arange(start, end+n, n)
    # checking that the array is not empty but stopping the code if it is empty
    if snap_ids.size == 0:
        print("no snapshots found (invalid input)")
        return

    # setting tolerance and VolDec for calculating COM_P in CenterOfMass
    delta = 0.1
    volDec = 4.0

    # looping over the txt files within each galaxy folder 
    # each folder is named as the galaxy name (MW, M31, or M33)
    for i, snap_id in enumerate(snap_ids):
        
        # composing the data filename
        ilbl = f"{snap_id:03d}"  # looks at the last three digits of the snapshot file
        filename = f"{galaxy}/{galaxy}_{ilbl}.txt" # folder/file
    
        # consider the ith particle (either disk or halo):
        # len(N) for N total particles in the galaxy component
        r = np.zeros(len(N))
        v = np.zeros(len(N))
        m = np.zeros(len(N))
        
        # need to loop through the number of particles in the component
        for j in range(len(N))
            r[j], v[j], m[j] = ParticleProperties(filename, type, N)

        # at any given time, we know com p and com v for the galaxy component (either disk or halo)
        # create an instance of CenterofMass class for galaxy with particle type
        galaxy_COM = CenterOfMass(filename, type)

        # storing the COM position and COM velocity
        r_COM = galaxy_COM.COM_P(delta, volDec)
        v_COM = galaxy_COM.COM_V(r_COM[0], r_COM[1], r_COM[2])

        # update the radii and velocities to be in the COM frame
        r_new = r - r_COM
        v_new = v - v_COM

        # calculate angular momentum for the ith particle
        AngularMomentum_i = m * np.cross(r_new, v_new)
        
        # total angular momentum for the galaxy component would be the sum of every particle's
        # angular momentum in that galaxy component
        AngularMomentum = np.sum(AngularMomentum_i)
    
        return AngularMomentum


In [ ]:
# can normalize the angular momentum vector since we only care about direction for this
# question

In [ ]:
# assign variables to the outputs of AngularMomentum for MW and M31 for their disk and
# halo components and for pre and post merger

# for MW:

# disk:

# pre-merger:

# halo:

# pre-merger:


# for M31:

# disk:

# pre-merger:

# halo:

# pre-merger:


# MW-M31 remnant:

# disk:

# halo:




# Prograde or Retrograde?
Because the angular momentum vectors are in 3D and the directions are not simply clockwise or counterclockwise, we must calculate the dot product between the two vectors to determine whether they oppose each other or not.
\begin{equation}
    \vec{L_{halo}} \cdot \vec{L_{disk}} = |\vec{L_{halo}}||\vec{L_{disk}}|cos\theta \rightarrow cos\theta = \frac{\vec{L_{halo}} \cdot \vec{L_{disk}}}{|\vec{L_{halo}}||\vec{L_{disk}}|}
\end{equation}
If the cosine term is negative, the orbit of the dark matter halo is prograde. If the cosine term is positive, the orbit of the dark matter halo is retrograde.

In [ ]:
# compute the dot product of the angular momenta



In [ ]:
# plot the dot product as a function of time

